In [ ]:
!pip install -q transformers accelerate evaluate ftfy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import torch

from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments
)

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

import evaluate
from tqdm.auto import tqdm


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

Device: cuda
GPU: Tesla T4


In [ ]:
df = pd.read_csv("garuda_news_preprocessed_final.csv")
df.head()

,link,judul,konten,tanggal,portal,tag,sentiment,konten_clean_final
0,https://kumparan.com/kumparanbisnis/garuda-ind...,Garuda Indonesia Kembali RUPSLB di Tengah Isu ...,Garuda Indonesia Kembali RUPSLB di Tengah Isu ...,30/09/2025,Kumparan,Manajemen,Neutral,Garuda Indonesia Kembali RUPSLB di Tengah Isu ...
1,https://www.bloombergtechnoz.com/detail-news/8...,Garuda Gelar RUPSLB di Tengah Isu Masuknya Dir...,Garuda Gelar RUPSLB di Tengah Isu Masuknya Dir...,29/09/2025,Bloomberg Technoz,Manajemen,Neutral,Garuda Gelar RUPSLB di Tengah Isu Masuknya Dir...
2,https://voi.id/ekonomi/519004/komisi-v-dpr-bak...,Komisi V DPR Bakal Dalami Dugaan Mafia Jual Be...,JAKARTA - Ketua Komisi V DPR Lasarus mengataka...,29/09/2025,VOI.ID,Rute/Operasional,Negative,JAKARTA - Ketua Komisi V DPR Lasarus mengataka...
3,https://www.kompasiana.com/zainularifin2714/68...,Rencana Merger Garuda Indonesia - Pelita Air: ...,"Latar Belakang\nPada pertengahan 2023, wacana ...",29/09/2025,Kompasiana.com,Lainnya,Neutral,"Latar Belakang Pada pertengahan 2023, wacana k..."
4,https://www.cnnindonesia.com/ekonomi/202509292...,Dony Oskaria Pastikan Merger Pelita Air-Garuda...,--\nPlt Menteri Badan Usaha Milik Negara (BUMN...,29/09/2025,CNN Indonesia,Lainnya,Neutral,-- Plt Menteri Badan Usaha Milik Negara (BUMN)...


In [ ]:
TEXT_COL = "konten_clean_final"   # <- ganti kalau kolommu namanya beda
LABEL_COL = "sentiment"           # <- ganti kalau labelmu beda (mis. sentiment/tag_new)

df = df.dropna(subset=[TEXT_COL, LABEL_COL]).copy()
df[TEXT_COL] = df[TEXT_COL].astype(str).str.strip()
df = df[df[TEXT_COL] != ""]
df[[TEXT_COL, LABEL_COL]].head()

,konten_clean_final,sentiment
0,Garuda Indonesia Kembali RUPSLB di Tengah Isu ...,Neutral
1,Garuda Gelar RUPSLB di Tengah Isu Masuknya Dir...,Neutral
2,JAKARTA - Ketua Komisi V DPR Lasarus mengataka...,Negative
3,"Latar Belakang Pada pertengahan 2023, wacana k...",Neutral
4,-- Plt Menteri Badan Usaha Milik Negara (BUMN)...,Neutral


In [ ]:
label2id = {"Negative": 0, "Neutral": 1, "Positive": 2}
id2label = {v: k for k, v in label2id.items()}

df["label"] = df[LABEL_COL].map(label2id)

# cek ada yang gagal mapping?
unmapped = df[df["label"].isna()][LABEL_COL].value_counts()
print("Unmapped labels:\n", unmapped)

df = df.dropna(subset=["label"]).copy()
df["label"] = df["label"].astype(int)

df["label"].value_counts()

Unmapped labels:
 Series([], Name: count, dtype: int64)


,count
label,
2,200
0,183
1,85


In [ ]:
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)

val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    random_state=42,
    stratify=temp_df["label"]
)

print("Train:", train_df.shape)
print("Val  :", val_df.shape)
print("Test :", test_df.shape)

print("\nLabel dist (train):\n", train_df["label"].value_counts(normalize=True))
print("\nLabel dist (val):\n", val_df["label"].value_counts(normalize=True))
print("\nLabel dist (test):\n", test_df["label"].value_counts(normalize=True))

Train: (374, 9)
Val  : (47, 9)
Test : (47, 9)

Label dist (train):
 label
2    0.427807
0    0.390374
1    0.181818
Name: proportion, dtype: float64

Label dist (val):
 label
2    0.425532
0    0.382979
1    0.191489
Name: proportion, dtype: float64

Label dist (test):
 label
2    0.425532
0    0.404255
1    0.170213
Name: proportion, dtype: float64


In [ ]:
from transformers import AutoTokenizer

MODEL_NAME = "indobenchmark/indobert-base-p2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
MAX_LEN = 256
TEXT_COL = "konten_clean_final"

In [ ]:
from transformers import AutoTokenizer

def tokenize_texts(texts):
    return tokenizer(
        texts.tolist(),
        truncation=True,
        padding=False,      # padding nanti pakai DataCollator (lebih efisien)
        max_length=MAX_LEN
    )

train_enc = tokenize_texts(train_df[TEXT_COL])
val_enc   = tokenize_texts(val_df[TEXT_COL])
test_enc  = tokenize_texts(test_df[TEXT_COL])

In [ ]:
import torch
from torch.utils.data import Dataset

class NewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels.tolist()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

train_dataset = NewsDataset(train_enc, train_df["label"])
val_dataset   = NewsDataset(val_enc,   val_df["label"])
test_dataset  = NewsDataset(test_enc,  test_df["label"])


In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#Training

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 3  # Negative, Neutral, Positive

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="macro", zero_division=0
    )
    return {
        "accuracy": acc,
        "precision_macro": precision,
        "recall_macro": recall,
        "f1_macro": f1
    }


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./runs/exp02_warmup_linear",
    eval_strategy="epoch",        # ⬅️ GANTI INI
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50,

    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,

    warmup_ratio=0.1,          # ✅ BARU: 10% langkah awal warmup
    lr_scheduler_type="linear",

    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,

    fp16=torch.cuda.is_available(),
    report_to="none"
)


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()


/tmp/ipython-input-2287156170.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision Macro,Recall Macro,F1 Macro
1,No log,0.879441,0.723404,0.482424,0.596296,0.533333
2,No log,0.524529,0.808511,0.767426,0.705556,0.696283
3,0.819400,0.475789,0.851064,0.840909,0.779630,0.790476
4,0.819400,0.506204,0.744681,0.645733,0.650000,0.642722
5,0.332600,0.618463,0.851064,0.846377,0.779630,0.791685
6,0.332600,0.533154,0.829787,0.790789,0.781481,0.784472
7,0.106100,0.607821,0.829787,0.790789,0.781481,0.784472
8,0.106100,0.663767,0.829787,0.790789,0.781481,0.784472
9,0.023900,0.712140,0.808511,0.760234,0.744444,0.747863
10,0.023900,0.719451,0.808511,0.760234,0.744444,0.747863


TrainOutput(global_step=240, training_loss=0.2682387142131726, metrics={'train_runtime': 418.1458, 'train_samples_per_second': 8.944, 'train_steps_per_second': 0.574, 'total_flos': 492022091151360.0, 'train_loss': 0.2682387142131726, 'epoch': 10.0})

In [ ]:
test_metrics = trainer.evaluate(test_dataset)
test_metrics

{'eval_loss': 0.7963907718658447,
 'eval_accuracy': 0.8085106382978723,
 'eval_precision_macro': 0.7594997594997595,
 'eval_recall_macro': 0.7473684210526316,
 'eval_f1_macro': 0.7517803517803517,
 'eval_runtime': 0.244,
 'eval_samples_per_second': 192.587,
 'eval_steps_per_second': 8.195,
 'epoch': 10.0}

In [ ]:
from sklearn.metrics import classification_report

pred = trainer.predict(test_dataset)
y_true = pred.label_ids
y_pred = np.argmax(pred.predictions, axis=1)

print(classification_report(y_true, y_pred, target_names=[id2label[0], id2label[1], id2label[2]]))

              precision    recall  f1-score   support

    Negative       0.89      0.84      0.86        19
     Neutral       0.57      0.50      0.53         8
    Positive       0.82      0.90      0.86        20

    accuracy                           0.81        47
   macro avg       0.76      0.75      0.75        47
weighted avg       0.80      0.81      0.81        47



#Function save experiment

In [ ]:
import os
import json
import pandas as pd
from datetime import datetime
import platform
import torch

def save_experiment_pretty(
    exp_name: str,
    model_name: str,
    text_col: str,
    max_len: int,
    train_size: int,
    val_size: int,
    test_size: int,
    label_dist_train: dict,
    label_dist_val: dict,
    label_dist_test: dict,
    training_args,      # TrainingArguments
    val_metrics: dict,  # trainer.evaluate(val_dataset)
    test_metrics: dict, # trainer.evaluate(test_dataset)
    extra_notes: str = "",
    out_dir: str = "experiments",
    summary_csv: str = "experiments_summary.csv"
):
    os.makedirs(out_dir, exist_ok=True)

    record = {
        "exp_name": exp_name,
        "timestamp": datetime.now().isoformat(),
        "env": {
            "python": platform.python_version(),
            "platform": platform.platform(),
            "torch_cuda": torch.cuda.is_available(),
            "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else None,
        },
        "data": {
            "text_col": text_col,
            "max_len": max_len,
            "sizes": {"train": train_size, "val": val_size, "test": test_size},
            "label_dist": {"train": label_dist_train, "val": label_dist_val, "test": label_dist_test}
        },
        "model": {"model_name": model_name},
        "tuning": training_args.to_dict(),   # semua hyperparam TrainingArguments
        "results": {"val": val_metrics, "test": test_metrics},
        "notes": extra_notes
    }

    # 1) simpan 1 file JSON rapi per eksperimen
    json_path = os.path.join(out_dir, f"{exp_name}.json")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(record, f, ensure_ascii=False, indent=2)

    # 2) append ringkasan ke CSV biar gampang bikin tabel tuning
    row = {
        "exp_name": exp_name,
        "timestamp": record["timestamp"],
        "model_name": model_name,
        "max_len": max_len,
        "epochs": record["tuning"].get("num_train_epochs"),
        "train_bs": record["tuning"].get("per_device_train_batch_size"),
        "eval_bs": record["tuning"].get("per_device_eval_batch_size"),
        "lr": record["tuning"].get("learning_rate"),
        "weight_decay": record["tuning"].get("weight_decay"),
        "val_f1_macro": val_metrics.get("eval_f1_macro"),
        "val_acc": val_metrics.get("eval_accuracy"),
        "test_f1_macro": test_metrics.get("eval_f1_macro"),
        "test_acc": test_metrics.get("eval_accuracy"),
        "notes": extra_notes,
        "json_path": json_path
    }

    summary_path = os.path.join(out_dir, summary_csv)
    df_row = pd.DataFrame([row])
    if os.path.exists(summary_path):
        df_row.to_csv(summary_path, mode="a", header=False, index=False, encoding="utf-8")
    else:
        df_row.to_csv(summary_path, index=False, encoding="utf-8")

    print(f"Saved: {json_path}")
    print(f"Updated summary: {summary_path}")


In [ ]:
val_metrics = trainer.evaluate(val_dataset)
test_metrics = trainer.evaluate(test_dataset)

label_dist_train = train_df["label"].value_counts(normalize=True).to_dict()
label_dist_val   = val_df["label"].value_counts(normalize=True).to_dict()
label_dist_test  = test_df["label"].value_counts(normalize=True).to_dict()

save_experiment_pretty(
    exp_name="exp02_warmup_linear",
    model_name=MODEL_NAME,
    text_col=TEXT_COL,
    max_len=MAX_LEN,
    train_size=len(train_df),
    val_size=len(val_df),
    test_size=len(test_df),
    label_dist_train=label_dist_train,
    label_dist_val=label_dist_val,
    label_dist_test=label_dist_test,
    training_args=training_args,
    val_metrics=val_metrics,
    test_metrics=test_metrics,
    extra_notes="Baseline fine-tuning (no discriminative LR)."
)


Saved: experiments/exp02_warmup_linear.json
Updated summary: experiments/experiments_summary.csv


In [ ]:
import pandas as pd
import os

EXP_NAME = "exp02_warmup_linear_log.csv"  # ganti tiap eksperimen
os.makedirs("logs", exist_ok=True)

pd.DataFrame(trainer.state.log_history).to_csv(
    f"logs/{EXP_NAME}_log.csv",
    index=False
)